# Goal of the project

The goal of this project is to control a 2D quadrotor to perform acrobatic moves. There are 4 parts of the project, where you will build controllers of increasing complexity. The last part will lead to the implementation of the iterative LQR (iLQR) algorithm.

## Instructions
Answer all the questions in the 4 parts below. You will need to submit:
1. A report (pdf format only - every other format will be rejected) answering all the questions that do not request code. DO NOT include code in the report.
2. One (or several) Jupyter notebook(s) containing all the code used to answer the questions. The notebook(s) should be runnable as is.

## 2D quadrotor

The quadrotor is depicted in the following figure <img src='quadrotor.png' width="300">
The quadrotor model is written as
$$\begin{align} 
\dot{x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $x$ is the horizontal and $y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $z = [x, v_x, y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines useful constants (mass, length, gravity, etc) and functions to simulate and animate the quadrotor as shown below.

## Part 1 - Setting up
1. Discretize the system dynamics using the method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)
2. Assume that the robot starts at an arbitrary position $x(0) = x_0$, $y(0) = y_0$ and $\theta(0) = 0$ with 0 velocities. Compute $u_1^*$ and $u_2^*$ such that the robot stays at this position forever after (you may test your answer using the simulation below).
3. Analyzing the system dynamics, is it possible to move in the x direction while keeping $\theta = 0$? Explain why.
4. Analyzing the system dynamics, is it possible to have the system at rest with $\theta = \frac{\pi}{2}$ (i.e. have the quadrotor in a vertical position)? Explain why.

## Part 2 - LQR to stay in place
Now that we have $u^*$ capable of keeping the robot at rest, we can design a simple controller that ensures that the robot stays in place even when pushed around by random disturbances (e.g. due to the wind). Our task here will be to design a LQR controller that keeps the robot at a predefined position. Since the dynamics is not linear, we need to compute a linear approximation of it.
1. Linearize the dynamics at an arbitrary operating point $z^*$, $u^*$ and write the linearized system dynamics using the variables $\bar{z}_n = z_n - z^*$ and $\bar{u}_n = u_n - u^*$.
2. Write a function ```get_linearization(z, u)``` that returns the matrices A and B given a state $z$ and a control $u$ (use the constants defined in the ``quadrotor.py`` module).
3. Using the linearized dynamics, we can design an infinite horizon LQR controller of the form, $\hat{u} = K \bar{z}$ to stabilize the resting point. Write the equations of the controller in the original coordinates $u$ as a function of $z$.
4. Design an infinite-horizon LQR controller that stabilizes the origin $z=0$ and test it using the simulator below.
5. Explain your intended design in the report, including the cost function and found control law. In particular, verify that it can handle perturbations by calling the ```simulate``` function with ```disturbance = True``` (when setting disturbance to ``True``, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state evolution and show the animation (include the plots in your report).

## Part 3 - following a trajectory using linearized dynamics
Now we want to follow a given trajectory leveraging a linearized version of the dynamics to design LQ controllers.
1. Assume that we want to follow a circle of radius 1 centered at (0,0) while keeping an orientation $\theta=\frac{\pi}{4}$, how does the linearization of the dynamics change along the desired trajectory? Why?
2. Design a tracking controller (using an LQ design with linear approximations) to follow this desired trajectory. Explain your design in the report. 
3. Test the tracking controller with the simulation (with and without the perturbations) and verify that you can indeed track the (x,y) trajectory very well. Are you able to also track $\theta$? (Explain) 
4. Analyze your results (including plots of the states, controls, etc). What benefits and issues do you see with this approach?
5. Answer questions 1-4 with a desired orientation of $\theta = 0$

## Part 4 - iterative LQR
Now we would like to do more complicated motions with the robot, like a flip. In this case, we do not have a prescribed trajectory but we would like to compute a locally optimal trajectory while we optimize the controller. We will use the *iterative LQR* algorithm to solve this problem.
### Task 1 - reaching a vertical orientation
In the first task, we want the robot to reach a vertical orientation $\theta = \frac{\pi}{2}$ at the location $x=3$ and $y=3$ at time $t=5$ starting from $z_0=0$. During the rest of the motion, the robot should try and stay close to the origin. It should also try to keep its control $u$ close to the control needed to keep the robot at rest. We want to make sure the robot reaches the origin $z=0$ at the end of the movement. 
1. Find a time-varying cost function that promotes such a behavior (use only quadratic/linear terms for simplicity). Assume $T=10$ seconds.
2. Write a function ```compute_cost(z,u, horizon_length)``` that returns the cost of a trajectory z with control trajectory u (using the cost function you wrote in question 1).
3. Compute the quadratic approximation of your cost function along an arbitrary trajectory of states $z_n$ with control trajectory $u_n$ (this is not just your cost function!)
4. Write a function ```get_quadratic_approximation_cost(z, u, horizon_length)``` that returns the quadratic approximation (Hessian matrices and Jacobians) of the cost function when approximated along the trajectory z with control trajectory u.
5. Write the iLQR algorithm that solves the problem using the functions written above. DO NOT FORGET the line search step at each iteration. For the line search, start with $\alpha = 1.$ and decrease it by half when the cost does not improve (you can stop when $\alpha < 0.01$).
6. Test the algorithm using as initial guess $u$ such that the robot is at rest (using the results of Part 1.2). Analyze your results (probably you will need to "tune" your cost function), plot the initial and final state and control trajectories, show the animation. Use the simulation without perturbations for simplicity. 
7. What benefits and issues do you see with this approach?
### Task 2 - doing a full flip
In the second task, we want the robot to do a full flip, trying to reach the upside-down state $x=1.5$, $y=3$ and $\theta = \pi$ at $t=5$ and upright state $x=3$, $y=0$ and $\theta = 2\pi$ at $T=10$.
8. Use iLQR (and a new cost function) to get the quadrotor to perform the task. Analyze your results. 
9. What benefits and issues do you see with this approach? Could you run the resulting controller on a real robot?

In [19]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
#from numpy import sin, cos, pi
import quadrotor
import sympy as sym

from sympy.utilities.lambdify import lambdify

In [20]:
# we can get its mass, half length (r), gravity constant
print(f'm is {quadrotor.MASS}')
print(f'r is {quadrotor.LENGTH}')
print(f'I is {quadrotor.INERTIA}')
print(f'g is {quadrotor.GRAVITY}')

# we can also get the integration step used in the simulation
print(f'dt is {quadrotor.DELTA_T}')

# we can get the size of its state and control vector
print(f'number of states {quadrotor.NUMBER_STATES} and number of controls {quadrotor.NUMBER_CONTROLS}')
print('the states are indexed as follows: x, vx, y, vy, theta, omega')

m is 0.6
r is 0.2
I is 0.15
g is 9.81
dt is 0.01
number of states 6 and number of controls 2
the states are indexed as follows: x, vx, y, vy, theta, omega


In [21]:
# we can simulate the robot but we need to provide a controller of the following form
x,V_x,y,V_y,theta,omega,u1,u2 = sym.symbols('x V_x y V_y theta omega u1 u2')
dt=quadrotor.DELTA_T
mass=quadrotor.MASS
Len=quadrotor.LENGTH
Inertia=quadrotor.INERTIA
grav=quadrotor.GRAVITY


np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

def get_linearization(z,u):
        # Description
        dt=quadrotor.DELTA_T
        mass=quadrotor.MASS
        length=quadrotor.LENGTH
        Ine=quadrotor.INERTIA
        grav=quadrotor.GRAVITY

        #_______________________
        x     = z[0]
        V_x   = z[1]
        y     = z[2]
        V_y   = z[3]
        theta = z[4]
        omega = z[5]

        u1 = u[0]
        print(u[1])
        u2 = u[1]
        print(u[1])
        #________________________

        #_________________________________________________________replace 1 and 2
        #z_next=quadrotor.get_next_state(z,u)
        #_____________________________________________________________replace 1 and 2

        #__________________________________1
        # Continuous Dynamics for Quadrotor in Symbolic
        A_pseudo = sym.Matrix([[V_x],[0],[V_y],[-grav],[omega],[0]])
        B_pseudo = sym.Matrix([[0],[(-sym.sin(theta)/mass)*(u1+u2)],[0],[(sym.cos(theta)/mass)*(u1+u2)],[0],[(length/Ine)*(u1-u2)]])  
        dzdt=A_pseudo+B_pseudo
        dzdt_T=dzdt.T
        #___________________________________1
        
        #___________________________________2
        # Discrete Dynamics for Quadrotor in Symbolic
        z_n_T= z.T
        z_next = z_n_T + (dt*dzdt_T)
        #___________________________________2

        #___________________________________3
        # Jacobian from the Discrete Dynamics in Symbolic and Numeric
        A_sym = z_next.jacobian(z)
        B_sym = z_next.jacobian(u)
        A_num = lambdify((x,V_x,y,V_y,theta,omega,u1,u2),A_sym,'numpy')
        B_num = lambdify((x,V_x,y,V_y,theta,omega,u1,u2),B_sym,'numpy')
        #____________________________________3

        return A_num,B_num

#make A,B,K global
z_n = sym.Matrix([[x],[V_x],[y],[V_y],[theta],[omega]])
u_n = sym.Matrix([[u1],[u2]])
A_out,B_out=get_linearization(z_n,u_n)


def dummy_controller(state, i):
    """
        the prototype of a controller is as follows
        state is a column vector containing the state of the robot
        i is the index corresponding to the time step in the horizon (useful to index gains K for e.g.)
        
        this controller needs to return an array of size (2,)
    """

    def solve_LQR_trajectory(A, B, Q, R, x_bar, N):
        """
        A, B, Q and R are the matrices defining the OC problem
        x_bar is the trajectory of desired states of size dim(x) x (N+1)
        N is the horizon length

        The function returns 1) a list of gains of length N and 2) a list of feedforward controls of length N
        """
        K_gains = []
        k_feedforward = []
        for i in range(0,N+1):
            if i == 0:
                PN=Q
                P=PN
                qN=-Q@x_bar[:,N-i] #N-i, allows us to start from the begining(N-i) and end at 0
                p=qN

            else:
                #Initializing transposed arrays
                BT=np.transpose(B)
                AT=np.transpose(A)
                #Initializing variables for clean code
                qn=-Q@x_bar[:,N-i] #N-i, allows us to start from the begining(N-i) and end at 0
                Y=-np.linalg.inv(R+(BT@P@B))
                Z=AT@P@B

                #Main meat
                K_n = Y @ (BT@P@A)
                P_n = Q + (AT@P@A) + (Z@K_n)
                k_n = Y @ (BT@p)
                p_n = qn + (AT@p) + (Z@k_n)

                #for the next iteration
                P = P_n
                p = p_n

                #Stores the values in front, thereby bypassing .append and .reverse
                K_gains.insert(0,K_n)
                k_feedforward.insert(0,k_n)
        return K_gains, k_feedforward

    u_n1=np.zeros(2)
    u_n1[0]=mass*grav*0.5
    u_n1[1]=mass*grav*0.5
    A=A_out
    B=B_out
    A1 = A(state[0],state[1],state[2],state[3],state[4],state[5],u_n1[0],u_n1[1])
    B1 = B(state[0],state[1],state[2],state[3],state[4],state[5],u_n1[0],u_n1[1])
    Q=np.eye(6)*100
    R=np.eye(2)*0.01
    N=1000

    #____________________________ z_bar
    t = np.arange(0.,10.01, dt)
    N1 = len(t)-1
    omega1 = 0.5*np.pi

    for ii in range(N1+1):
        omegaT=omega1*t[ii]
        #x_bar1 [position for 1st dimension, derivative of position for 1st dimension, position for 2nd dimension, derivative of position for 2nd dimension]
        z_bar1 = np.array([[np.cos(omegaT)],[-omega1 * np.sin(omegaT)], [np.sin(omegaT)],[omega1 * np.cos(omegaT)],[0],[0]])
        if ii == 0:
            z_bar=z_bar1
        else:
            z_bar = np.hstack((z_bar,z_bar1))
    #_________________________________

    K_gains,k_feedforward = solve_LQR_trajectory(A1,B1,Q,R,z_bar,N)

    x_n=state.reshape(len(state),1) 
    #K_n=K[i][:][:]   #Initializing necessary variable to store K_n
    K_n=K_gains[i]
    k_n=k_feedforward[i]
    k_n=k_n.reshape(len(k_n),1)
    u_n=(K_n @ x_n) + k_n #Formula from Prof L. Righetti
    print(u_n.shape)
    u=u_n.reshape(len(u_n),) #reshape of u/u_n to allow for stacking

    # here we do nothing and just return some non-zero control
    return u




# we can now simulate for a given number of time steps - here we do 10 seconds
horizon_length = 1000
z0 = np.zeros([quadrotor.NUMBER_STATES,])
t, state, u = quadrotor.simulate(z0, dummy_controller, horizon_length, disturbance = False)

u2
u2
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)


In [22]:
# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0.5, 0, 'Time [s]')

In [23]:
# now we can also create an animation
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.

In [24]:
# we can also simulate with perturbations
t, state, u = quadrotor.simulate(z0, dummy_controller, horizon_length, disturbance = True)

# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

quadrotor.animate_robot(state,u)

(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)
(2, 1)
(2, 6)
(2, 1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Your browser does not support the video tag.